# Neural network to approximate Runge's function

In [1]:
# Imports
import autograd.numpy as np  # We need to use this numpy wrapper to make automatic differentiation work later
from autograd import grad, elementwise_grad
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import (
    PolynomialFeatures,
)
import json

from functions import runge, OLS_parameters, MSE
from functions import ReLU, ReLU_der, sigmoid, sigmoid_der, leaky_ReLU, leaky_ReLU_der, mse_der
from functions import MSE, identity, identity_der, R2
from nn_class import NeuralNetwork

In [2]:
import os

with open("data/xy.json", "r") as f:
    data = json.load(f)

# reconstruct arrays with same shape as originally saved
x = np.array(data["x"]).reshape(-1, 1)
y = np.array(data["y"]).reshape(-1, 1)

# recuperate dimensions
n_samples, n_features = x.shape
print("Loaded shapes -> x:", x.shape, "y:", y.shape)
print("n_samples:", n_samples, "n_features:", n_features)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y_offset = y_train.mean()

poly = PolynomialFeatures(degree=10)
X_train = poly.fit_transform(x_train)
X_test = poly.fit_transform(x_test)
# scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

Loaded shapes -> x: (1000, 1) y: (1000, 1)
n_samples: 1000 n_features: 1


In [ ]:
possible_layer_sizes = [50, 100]

possible_number_of_layers = [1, 2]

possible_activations = [(ReLU, ReLU_der), (leaky_ReLU, leaky_ReLU_der)]

possible_optimizers = [None, 'RMSPROP', 'ADAM']

scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)
inputs = x_train_s
targets = y_train


for activation, activation_der in possible_activations:
    for n_layers in possible_number_of_layers:
        for n_neurons in possible_layer_sizes:
            layer_output_sizes = [n_neurons] * n_layers + [1]
            activation_funcs = [activation] * n_layers + [identity]
            activation_ders = [activation_der] * n_layers + [identity_der]

            NN = NeuralNetwork(
                x_train_s,
                targets,
                layer_output_sizes,
                activation_funcs,
                activation_ders,
                MSE,
                mse_der,
                L2=True,
                lmbda=0.01,
            )

            for optimizer in possible_optimizers:
                if optimizer is None:
                    NN.train_network_stochastic_gd(epochs=10000)
                else:
                    NN.train_network_stochastic_gd(epochs=10000, lr_method=optimizer)

                targets = y_test

                predictions = NN.predict(x_test_s)
                print('NN mse:', MSE(predictions, targets))
                print('NN R2:', R2(predictions, targets))

                mse_val = MSE(predictions, targets)
                R2_val = R2(predictions, targets)

                opt_name = optimizer if optimizer is not None else "sgd"
                metrics = {"mse": float(mse_val), "r2": float(R2_val)}
                filename = f"data/nn_sgd_{n_neurons}_{n_layers}_{activation.__name__}_{opt_name}_l2_0_01.json"
                with open(filename, "w") as f:
                    json.dump(metrics, f, indent=2)

In [ ]:
possible_layer_sizes = [50, 100]

possible_number_of_layers = [1, 2]

possible_activations = [(ReLU, ReLU_der), (leaky_ReLU, leaky_ReLU_der)]

possible_optimizers = [None, 'RMSPROP', 'ADAM']

scaler.fit(x_train)
x_train_s = scaler.transform(x_train)
x_test_s = scaler.transform(x_test)
inputs = x_train_s
targets = y_train


for activation, activation_der in possible_activations:
    for n_layers in possible_number_of_layers:
        for n_neurons in possible_layer_sizes:
            layer_output_sizes = [n_neurons] * n_layers + [1]
            activation_funcs = [activation] * n_layers + [identity]
            activation_ders = [activation_der] * n_layers + [identity_der]

            NN = NeuralNetwork(
                x_train_s,
                targets,
                layer_output_sizes,
                activation_funcs,
                activation_ders,
                MSE,
                mse_der,
                L2=True,
                lmbda=0.01,
            )

            for optimizer in possible_optimizers:
                if optimizer is None:
                    NN.train_network_plain_gd()
                else:
                    NN.train_network_plain_gd(lr_method=optimizer)

                targets = y_test

                predictions = NN.predict(x_test_s)
                print('NN mse:', MSE(predictions, targets))
                print('NN R2:', R2(predictions, targets))

                mse_val = MSE(predictions, targets)
                R2_val = R2(predictions, targets)

                opt_name = optimizer if optimizer is not None else "sgd"
                metrics = {"mse": float(mse_val), "r2": float(R2_val)}
                filename = f"data/nn_plain_{n_neurons}_{n_layers}_{activation.__name__}_{opt_name}_l2_0_01.json"
                with open(filename, "w") as f:
                    json.dump(metrics, f, indent=2)

NN mse: 0.011049828568476965
NN R2: 0.8661610462439043
